In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.datasets import load_breast_cancer

import scipy

In [ ]:
data = load_breast_cancer(return_X_y=False)

In [ ]:
X = pd.DataFrame(data=data.data, columns=data.feature_names)
y = data.target
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
means = X_train.groupby(y_train).apply(np.mean)
stds = X_train.groupby(y_train).apply(np.std)

In [ ]:
probabilities = X_train.groupby(y_train).apply(lambda x: len(x)) / X_train.shape[0]

**Bayes Formula**

$P(y|X) = \frac{P(X|y)\cdot P(y)}{P(X)}$

Since $P(X)$ is constant:

$P(y|X) = P(X|y)\cdot P(y)$

Given all that, we have: $P(y|X) = P(y)\cdot \prod^{N}_{i=1}P(x_i|y)$, so

$$\hat{y} = argmax P(y)\cdot \prod^{N}_{i=1}P(x_i|y)$$

Here we assume normal distribution of features and their independency: $P(x_i|y) = \frac{1}{\sqrt{2\cdot \pi \cdot \sigma_y^2}}\cdot \exp(-\frac{(x_i - \mu_y)^2}{2\cdot \sigma_y^2})$

In [ ]:
predictions = []

for element in range(X_test.shape[0]):  # for every element in test set
    prob = {}

    for cls in np.unique(y_train):  # for every given class label
        prob[cls] = probabilities.iloc[cls]  # apriori probability of class label (from train data)

        for index, feature in enumerate(X_test.iloc[element]):
            # probability of being drawn from the same distribution
            prob[cls] *= scipy.stats.norm.pdf(feature, means.iloc[cls, index], stds.iloc[cls, index])   
        # most probable class label
    pred = pd.Series(data=prob).values.argmax()
    predictions.append(pred)

In [ ]:
# accuracy
print(classification_report(y_test, predictions, digits=4))

              precision    recall  f1-score   support

           0     0.9756    0.9302    0.9524        43
           1     0.9589    0.9859    0.9722        71

    accuracy                         0.9649       114
   macro avg     0.9673    0.9581    0.9623       114
weighted avg     0.9652    0.9649    0.9647       114

